## Imports

In [30]:
import torch
import torch_sparse

FileNotFoundError: Could not find module 'C:\Users\valmi\miniconda3\envs\pytorch\Lib\site-packages\torch_sparse\_convert_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.

In [11]:
!nvidia-smi

Thu Dec 21 18:57:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:C4:00.0 Off |                    0 |
| N/A   57C    P0    86W / 300W |  77321MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Parameters

In [12]:
batch_size = 32
num_neurons = 100000
synapses_per_neuron = 1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Create a dense vector
indices = []
for i in range(num_neurons):
    synapses = torch.randint(0, num_neurons, (synapses_per_neuron,))
    synapse_root = torch.ones_like(synapses) * i
    indices.append(torch.stack((synapses, synapse_root)))
indices = torch.cat(indices, dim=1).to(device)
values = torch.randn(num_neurons*synapses_per_neuron).to(device)

indices, values = torch_sparse.coalesce(indices, values, num_neurons, num_neurons)
        
coo_matrix = torch.sparse_coo_tensor(indices, values, (num_neurons, num_neurons)).coalesce().to(device)
csr_matrix = coo_matrix.to_sparse_csr().to(device)
# dense_matrix = coo_matrix.to_dense().to(device)
dense_vector_batched = torch.randn(batch_size, num_neurons).to(device).t()
bias = torch.randn(1, num_neurons).t()

: 

## On GPU

In [31]:
%timeit torch.mm(coo_matrix, dense_vector_batched) + bias

508 µs ± 237 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [32]:
%timeit torch.mm(csr_matrix, dense_vector_batched)

404 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
%timeit torch.sparse.mm(coo_matrix, dense_vector_batched)

508 µs ± 444 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
%timeit torch.sparse.mm(csr_matrix, dense_vector_batched)

404 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
%timeit torch_sparse.spmm(indices, values, num_neurons, num_neurons, dense_vector_batched)

844 µs ± 568 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [36]:
# %timeit torch.mv(dense_matrix, dense_vector_batched)

## On CPU

In [17]:
coo_matrix = coo_matrix.to('cpu')
csr_matrix = csr_matrix.to('cpu')
# dense_matrix = dense_matrix.to('cpu')
dense_vector_batched = dense_vector_batched.to('cpu')
indices = indices.to('cpu')
values = values.to('cpu')

In [18]:
%timeit torch.mv(coo_matrix, dense_vector_batched)

17.1 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%timeit torch.mv(csr_matrix, dense_vector_batched)

1.23 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%timeit torch.sparse.mm(coo_matrix, dense_vector_batched)

18.3 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit torch.sparse.mm(csr_matrix, dense_vector_batched)

2.04 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit torch_sparse.spmm(indices, values, num_neurons, num_neurons, dense_vector_batched)

8 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# %timeit torch.mv(dense_matrix, dense_vector_batched)